In [41]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [48]:
r= requests.get('https://en.wikipedia.org/wiki/Toy_Story_3')
wiki= bs(r.content)

## 1. Scrapping the infobox on the Toy Story 3  Wiki Page

In [49]:
info=wiki.find('table', attrs={'class':'infobox vevent'})
info_rows=info.find_all('tr')

In [4]:
def get_content_value(x):
    if x.find('li'):
        return [y.get_text(' ',strip=True).replace('\xa0',' ') for y in x.find_all('li')]
    else:
        return x.get_text(' ',strip=True).replace('\xa0',' ')
    
movie_info = {}

for index,element in enumerate(info_rows):
    if index == 0:
        movie_info['Title']= element.find('th').get_text(' ',strip=True)
    elif index == 1:
        continue
    else:
        content_key= element.find('th').get_text(' ',strip=True)
        content_value= get_content_value(element.find('td'))
        movie_info[content_key]= content_value
        
movie_info

{'Title': 'Toy Story 3',
 'Directed by': 'Lee Unkrich',
 'Screenplay by': 'Michael Arndt',
 'Story by': ['John Lasseter', 'Andrew Stanton', 'Lee Unkrich'],
 'Produced by': 'Darla K. Anderson',
 'Starring': ['Tom Hanks',
  'Tim Allen',
  'Joan Cusack',
  'Don Rickles',
  'Wallace Shawn',
  'John Ratzenberger',
  'Estelle Harris',
  'Ned Beatty',
  'Michael Keaton',
  'Jodi Benson',
  'John Morris'],
 'Cinematography': ['Jeremy Lasky', 'Kim White'],
 'Edited by': 'Ken Schretzmann',
 'Music by': 'Randy Newman',
 'Production companies': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release dates': ['June 12, 2010 ( 2010-06-12 ) ( Taormina Film Fest )',
  'June 18, 2010 ( 2010-06-18 ) (United States)'],
 'Running time': '103 minutes [1]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$200 million [1]',
 'Box office': '$1.067 billion [1]'}

## 2. Scrapping All the Movies'infobox from Wiki Diney Movies List

In [47]:
r= requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
wiki2=bs(r.content)

In [7]:
movies= wiki2.select('.wikitable.sortable i')
movies[0].a['href']

'/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)'

In [8]:
def get_content_value(x):
    if x.find('li'):
        return [y.get_text(' ',strip=True).replace('\xa0',' ') for y in x.find_all('li')]
    elif x.find('br'):
        return [text for text in x.stripped_strings]
        
    else:
        return x.get_text(' ',strip=True).replace('\xa0',' ')
    
    
def clear_tag(links):
    for x in links.find_all(['sup','span']):
        x.decompose()

def get_infobox(url):
    
    r= requests.get(url)
    links= bs(r.content)
    info=links.find('table', attrs={'class':'infobox vevent'})
    info_rows=info.find_all('tr')

    clear_tag(links)

    movie_info = {}

    for index,element in enumerate(info_rows):
        if index == 0:
            movie_info['Title']= element.find('th').get_text(' ',strip=True)
        
        else:
            header=element.find('th')
            if header:
                content_key= element.find('th').get_text(' ',strip=True)
                content_value= get_content_value(element.find('td'))
                movie_info[content_key]= content_value
    
    return movie_info

In [9]:
get_infobox('https://en.wikipedia.org/wiki/One_Little_Indian_(film)')

{'Title': 'One Little Indian',
 'Directed by': 'Bernard McEveety',
 'Written by': 'Harry Spalding',
 'Produced by': 'Winston Hibler',
 'Starring': ['James Garner',
  'Vera Miles',
  'Pat Hingle',
  "Clay O'Brien",
  'John Doucette',
  'Morgan Woodward',
  'Andrew Prine'],
 'Cinematography': 'Charles F. Wheeler',
 'Edited by': 'Robert Stafford',
 'Music by': 'Jerry Goldsmith',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['June 20, 1973'],
 'Running time': '90 Minutes',
 'Country': 'United States',
 'Language': 'English',
 'Box office': '$2 million'}

In [10]:
r= requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
wiki2=bs(r.content)
movies= wiki2.select('.wikitable.sortable i a')

base_path="https://en.wikipedia.org/"

movie_lists=[]

for index,movie in enumerate(movies):
    
    
    try:
        site = movie['href']
        full_path = base_path + site
        
        
        movie_lists.append(get_infobox(full_path))
    
    except Exception as e:
        print(movie.get_text())
        print(e)

Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
The Beatles: Get Back – The Rooftop Concert
'NoneType' object has no attribute 'find'
Elemental
'NoneType' object has no attribute 'find_all'
Wish
'NoneType' object has no attribute 'find_all'
Elio
'NoneType' object has no attribute 'find_all'
61
'NoneType' object has no attribute 'find_all'
All Night Long
'NoneType' object has no attribute 'find'
Big Thunder Mountain Railroad
'NoneType' object has no attribute 'find_all'
Keeper of the Lost Cities
'NoneType' object has no attribute 'find_all'
Muppet Man
'NoneType' object has no attribute 'find_all'
One Thousand and One Nights
'NoneType' object has no attribute 'find_all'
Shrunk
'NoneType' object has no attribute 'find'
Sister Act 3
'NoneType' object has no attribute 'find'
The Graveyard Book
'NoneType' object has no attribute 'find_all'
The Thief
'NoneType' object has no attribute 'find_all'
Tom Sawyer
'NoneType' obje

In [11]:
len(movie_lists)

523

In [13]:
# Checkpoint

import json

def save_data(title,data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
def load_data(title):
    with open(title, 'w', encoding='utf-8') as f:
        return json.load(f)


In [14]:
save_data('Disney Movies Dataset(without cleaning).json', movie_lists)

## 3. Data Cleaning

- Cleaning the reference link and duplicated date
- Fixing the name tags(long strings)
- Looking into the error that were generated via creation of the movielist
- Taking the numeric value only from the Running Time Col
- Converting the budget and box office amount to proper numberic values
- Converting the Date to Datetime

In [15]:
def mins_to_int(x):
    if x == 'N/A':
        return None
    
    if isinstance(x,list):
        entry=x[0]
        time= int(entry.split(' ')[0])
        return time
    else:
        
        time= int(x.split(' ')[0])
        return time

for x in movie_lists:
    x['Running Time Fixed']= mins_to_int(x.get('Running time','N/A'))

In [16]:
print([x.get('Running Time Fixed')for x in movie_lists])

[83, 88, 126, 74, 64, 70, 42, 65, 71, 75, 94, 73, 75, 82, 68, 74, 96, 75, 84, 77, 92, 69, 81, 60, 127, 93, 76, 75, 73, 85, 81, 70, 90, 80, 75, 84, 83, 72, 97, 75, 104, 93, 105, 95, 97, 134, 69, 92, 126, 79, 97, 128, 73, 91, 105, 98, 130, 89, 93, 67, 98, 100, 118, 103, 110, 80, 74, 91, 91, 97, 118, 139, 131, 92, 87, 116, 93, 110, 110, 131, 101, 108, 84, 78, 75, 164, 106, 110, 99, 113, 108, 102, 85, 91, 93, 100, 100, 79, 96, 113, 89, 117, 92, 88, 92, 87, 93, 93, 93, 90, 83, 96, 88, 89, 91, 93, 92, 97, 100, 100, 89, None, 91, 112, 115, 95, 91, 97, 104, 74, 48, 77, 104, 128, 101, 94, 104, 90, 100, 88, 93, 98, 112, 84, 97, 97, 114, 96, 97, 109, 83, 90, 107, 96, 103, 91, 95, 105, 113, 80, 101, 90, 74, 90, 89, 110, 74, 93, 84, 83, 74, 77, 107, 93, 88, 108, 84, 121, 89, 104, 90, 86, 84, 108, 107, 96, 98, 105, 108, 94, 106, 102, 88, 102, 102, 97, 111, 100, 96, 96, 78, 81, 108, 89, 99, 89, 81, 92, 100, 89, 79, 91, 101, 104, 103, 86, 106, 74, 93, 92, 98, 95, 93, 87, 93, 87, 128, 77, 86, 95, 114, 

In [17]:
pattern_number= r'\d+(,\d{3})*\.*\d*'
pattern_amount= r'thousand|million|billion'


value_convert = rf'\${pattern_number}'
word_convert= rf'\$({pattern_number})(-|\sto\s|-)?({pattern_number})?\s({pattern_amount})'

def word_to_value(word):
    V={'thousand':1000,'million':1000000,'billion':1000000000}
    return V[word]

def main_method(x):
    value = re.search(pattern_number,x).group()
    value1 = float(value.replace(',',''))
    word = re.search(pattern_amount,x, flags=re.I).group().lower()
    word_value = word_to_value(word)
    total = word_value * value1
    return total


def method2(x):
    value = re.search(pattern_number,x).group()
    value1 = float(value.replace(',',''))
    return value1

def value_conversion(money):
    
    if money == 'N/A':
        return None
    
    if isinstance(money,list):
        money = money[0]
    
    money_syntax=re.search(value_convert, money)
    word_syntax =re.search(word_convert, money,flags=re.I)
    
    
    if word_syntax:
        return main_method(word_syntax.group())
    
    elif money_syntax:
        return method2(money_syntax.group())
    else:
        return None

In [18]:
for x in movie_lists:
    x['Budget_Fixed']= value_conversion(x.get('Budget', 'N/A'))
    x['Box office_Fixed']= value_conversion(x.get('Box office','N/A'))

In [19]:
print([x.get('Budget_Fixed','N/A')for x in movie_lists])

[1490000.0, 2600000.0, 2280000.0, 600000.0, 950000.0, 858000.0, None, 788000.0, None, 1350000.0, 2125000.0, None, 1500000.0, 1500000.0, None, 2200000.0, 1800000.0, 3000000.0, None, 4000000.0, 2000000.0, 300000.0, 1800000.0, None, 5000000.0, None, 4000000.0, None, None, None, None, None, None, 700000.0, None, None, None, None, None, 6000000.0, 1000000.0, None, 2000000.0, None, None, 2500000.0, None, None, 4000000.0, 3600000.0, None, None, None, None, 3000000.0, None, 3000000.0, None, None, None, None, None, None, None, None, None, 3000000.0, None, None, None, None, 4.4, None, None, None, None, None, None, None, None, None, None, None, 4000000.0, None, 5000000.0, None, None, None, None, 5000000.0, None, None, None, None, None, None, 4000000.0, None, None, None, 6300000.0, None, None, None, None, None, None, None, None, 5000000.0, None, None, None, None, 8000000.0, None, None, None, None, None, None, 1000000.0, None, None, None, None, 5000000.0, None, None, None, 7500000.0, None, 10000000

In [22]:
print([x.get('Box office_Fixed','N/A')for x in movie_lists])

[418000000.0, 164000000.0, 83300000.0, 960000.0, 1300000.0, 267399999.99999997, 1135000.0, 799000.0, 3355000.0, 3275000.0, 65000000.0, 3165000.0, 2560000.0, 3700000.0, 1625000.0, 182000000.0, 4100000.0, 2400000.0, 2100000.0, 87400000.0, 1000000.0, 2600000.0, None, 1750000.0, 28200000.0, 2150000.0, 187000000.0, 2100000.0, 1600000.0, 1700000.0, None, None, 2750000.0, None, 1750000.0, 6250000.0, None, 1800000.0, 2500000.0, 51600000.0, 12300000.0, None, 1700000.0, 3100000.0, None, 3750000.0, 2300000.0, None, 40000000.0, 303000000.0, 11426000.0, 25100000.0, None, None, 4600000.0, 3500000.0, 5000000.0, None, None, None, 21745500.0, 22100000.0, 2550000.0, 3000000.0, 4350000.0, 4200000.0, 22200000.0, 1600000.0, 4000000.0, 2250000.0, 3500000.0, 103100000.0, 3500000.0, 1275000.0, 4000000.0, 28068222.0, 6200000.0, 22565634.0, None, 16207116.0, 3000000.0, 1900000.0, 4000000.0, 378000000.0, None, 5000000.0, 21540050.0, 2250000.0, 4150000.0, 3300000.0, 51300000.0, 1300000.0, None, 5500000.0, None, 1

In [23]:
from datetime import datetime

dates = [x.get('Release date','N/A') for x in movie_lists]

def clean_date(date):
    return date.split('(')[0].strip()
    

def convert_date(date):
    if isinstance(date,list):
        date=date[0]
        
    if date == 'N/A':
        return None
    
    final_date= clean_date(date)
   
   
    fmts = ['%B %d, %Y','%d %B %Y']
    for fmt in fmts:
        try:
            return datetime.strptime(final_date, fmt)
        except:
            pass
    return None

In [24]:
for x in movie_lists:
    x['Release time fmt']=convert_date(x.get('Release date','N/A'))

In [25]:
print([x.get('Release time fmt','N/A')for x in movie_lists])

[None, None, datetime.datetime(1940, 11, 13, 0, 0), datetime.datetime(1941, 6, 27, 0, 0), None, None, None, datetime.datetime(1943, 7, 17, 0, 0), None, None, None, datetime.datetime(1947, 9, 27, 0, 0), datetime.datetime(1948, 5, 27, 0, 0), None, datetime.datetime(1949, 10, 5, 0, 0), None, None, None, None, datetime.datetime(1953, 2, 5, 0, 0), datetime.datetime(1953, 7, 23, 0, 0), datetime.datetime(1953, 11, 10, 0, 0), None, datetime.datetime(1954, 8, 17, 0, 0), datetime.datetime(1954, 12, 23, 0, 0), datetime.datetime(1955, 5, 25, 0, 0), datetime.datetime(1955, 6, 22, 0, 0), datetime.datetime(1955, 9, 14, 0, 0), datetime.datetime(1955, 12, 22, 0, 0), datetime.datetime(1956, 6, 8, 0, 0), datetime.datetime(1956, 7, 18, 0, 0), datetime.datetime(1956, 9, 4, 0, 0), datetime.datetime(1956, 12, 20, 0, 0), datetime.datetime(1957, 6, 19, 0, 0), datetime.datetime(1957, 8, 28, 0, 0), datetime.datetime(1957, 12, 25, 0, 0), datetime.datetime(1958, 7, 8, 0, 0), datetime.datetime(1958, 8, 12, 0, 0), d

In [28]:
movie_lists[2]

{'Title': 'Fantasia',
 'Directed by': ['Samuel Armstrong',
  'James Algar',
  'Bill Roberts',
  'Paul Satterfield',
  'Ben Sharpsteen',
  'David D. Hand',
  'Hamilton Luske',
  'Jim Handley',
  'Ford Beebe',
  'T. Hee',
  'Norman Ferguson',
  'Wilfred Jackson'],
 'Story by': ['Joe Grant', 'Dick Huemer'],
 'Produced by': ['Walt Disney', 'Ben Sharpsteen'],
 'Starring': ['Leopold Stokowski', 'Deems Taylor'],
 'Narrated by': 'Deems Taylor',
 'Cinematography': 'James Wong Howe',
 'Music by': 'See program',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['November 13, 1940'],
 'Running time': '126 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.28 million',
 'Box office': '$76.4–$83.3 million (United States and Canada)',
 'Running Time Fixed': 126,
 'Budget_Fixed': 2280000.0,
 'Box office_Fixed': 83300000.0,
 'Release time fmt': datetime.datetime(1940, 11, 13, 0, 0)}

In [30]:
# Checkpoint2 
# Had to use pickle over json as datetime cant be stored in json

import pickle

def save_data_pickle(name,data):
    with open(name,'wb') as f:
        pickle.dump(data,f)
        
def load_data_pickle(name):
    with open(name,'rb') as f:
        return pickle.load(f)

In [31]:
save_data_pickle('Disney Movies Dataset(Cleaned).pickle',movie_lists)

## 4. Attaching Review Scores(Rotten Tomatoes, Metascore, IMDB) with move database api

In [32]:
import requests
import os
import urllib

def omdb(title):
    base_url ='http://www.omdbapi.com/?'
    parameters = {'apikey': os.environ['OMDB_API_KEY'],'t':title}
    parameters_encode = urllib.parse.urlencode(parameters)
    full_url= base_url + parameters_encode
    
    return requests.get(full_url).json()

def rotten_tomato(info):
    ratings = info.get('Ratings',[])
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
    return None

In [33]:
for x in movie_lists:
    title=x['Title']
    omdb_info=omdb(title)
    x['imdb']=omdb_info.get('imdbRating',None)
    x['metascore']=omdb_info.get('Metascore',None)
    x['rotten_tomatoes']= rotten_tomato(omdb_info)

In [34]:
movie_lists[2]

{'Title': 'Fantasia',
 'Directed by': ['Samuel Armstrong',
  'James Algar',
  'Bill Roberts',
  'Paul Satterfield',
  'Ben Sharpsteen',
  'David D. Hand',
  'Hamilton Luske',
  'Jim Handley',
  'Ford Beebe',
  'T. Hee',
  'Norman Ferguson',
  'Wilfred Jackson'],
 'Story by': ['Joe Grant', 'Dick Huemer'],
 'Produced by': ['Walt Disney', 'Ben Sharpsteen'],
 'Starring': ['Leopold Stokowski', 'Deems Taylor'],
 'Narrated by': 'Deems Taylor',
 'Cinematography': 'James Wong Howe',
 'Music by': 'See program',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['November 13, 1940'],
 'Running time': '126 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.28 million',
 'Box office': '$76.4–$83.3 million (United States and Canada)',
 'Running Time Fixed': 126,
 'Budget_Fixed': 2280000.0,
 'Box office_Fixed': 83300000.0,
 'Release time fmt': datetime.datetime(1940, 11, 13, 0, 0),
 'imdb': '7.7',
 'metascore': '96',

In [35]:
# Checkpoint 3

save_data_pickle('Disney Movies Dataset(Finalized).pickle',movie_lists)

In [36]:
movie_lists_copy=[x.copy() for x in movie_lists]

In [37]:
for x in movie_lists_copy:
    current=x['Release time fmt']
    if current:
        x['Release time fmt']=current.strftime('%B %d,%Y')
    else:
        x['Release time fmt']= None

In [38]:
movie_lists_copy[2]

{'Title': 'Fantasia',
 'Directed by': ['Samuel Armstrong',
  'James Algar',
  'Bill Roberts',
  'Paul Satterfield',
  'Ben Sharpsteen',
  'David D. Hand',
  'Hamilton Luske',
  'Jim Handley',
  'Ford Beebe',
  'T. Hee',
  'Norman Ferguson',
  'Wilfred Jackson'],
 'Story by': ['Joe Grant', 'Dick Huemer'],
 'Produced by': ['Walt Disney', 'Ben Sharpsteen'],
 'Starring': ['Leopold Stokowski', 'Deems Taylor'],
 'Narrated by': 'Deems Taylor',
 'Cinematography': 'James Wong Howe',
 'Music by': 'See program',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['November 13, 1940'],
 'Running time': '126 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.28 million',
 'Box office': '$76.4–$83.3 million (United States and Canada)',
 'Running Time Fixed': 126,
 'Budget_Fixed': 2280000.0,
 'Box office_Fixed': 83300000.0,
 'Release time fmt': 'November 13,1940',
 'imdb': '7.7',
 'metascore': '96',
 'rotten_tomatoes'

In [39]:
save_data("Web Scrapping Project(Disney Movies'Dataset.json",movie_lists_copy)

## 5. Converting the Dictionary into dataframe and export it as csv

In [42]:
df=pd.DataFrame(movie_lists)

In [43]:
df.to_csv("Web Scraping Project(Disney Movies'Dataset).csv")

In [44]:
df.head()

,Title,Directed by,Written by,Based on,Produced by,Starring,Music by,Production company,Distributed by,Release dates,...,Traditional,Simplified,Original title,Layouts by,Music,Lyrics,Book,Basis,Productions,Awards
0,Snow White and the Seven Dwarfs,"[David Hand, William Cottrell, Wilfred Jackson...","[Ted Sears, Richard Creedon, Otto Englander, D...","[Snow White, by The, Brothers Grimm]",Walt Disney,"[Adriana Caselotti, Lucille La Verne, Harry St...","[Frank Churchill, Paul Smith, Leigh Harline]",Walt Disney Productions,RKO Radio Pictures,"[December 21, 1937 ( Carthay Circle Theatre ),...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pinocchio,"[Ben Sharpsteen, Hamilton Luske, Bill Roberts,...",NaN,"[The Adventures of Pinocchio, by, Carlo Collodi]",Walt Disney,"[Cliff Edwards, Dickie Jones, Christian Rub, W...","[Leigh Harline, Paul J. Smith]",Walt Disney Productions,RKO Radio Pictures,"[February 7, 1940 ( Center Theatre ), February...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fantasia,"[Samuel Armstrong, James Algar, Bill Roberts, ...",NaN,NaN,"[Walt Disney, Ben Sharpsteen]","[Leopold Stokowski, Deems Taylor]",See program,Walt Disney Productions,RKO Radio Pictures,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,The Reluctant Dragon,"[Alfred Werker, (live action), Hamilton Luske,...","[Live-action:, Ted Sears, Al Perkins, Larry Cl...",NaN,Walt Disney,"[Robert Benchley, Frances Gifford, Buddy Peppe...","[Frank Churchill, Larry Morey]",Walt Disney Productions,RKO Radio Pictures,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dumbo,"[Ben Sharpsteen, Norman Ferguson, Wilfred Jack...",NaN,"[Dumbo, the Flying Elephant, by, Helen Aberson...",Walt Disney,"[Edward Brophy, Verna Felton, Cliff Edwards, H...","[Frank Churchill, Oliver Wallace]",Walt Disney Productions,RKO Radio Pictures,"[October 23, 1941 (New York City), October 31,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df.shape

(523, 46)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Title                 523 non-null    object        
 1   Directed by           519 non-null    object        
 2   Written by            213 non-null    object        
 3   Based on              289 non-null    object        
 4   Produced by           510 non-null    object        
 5   Starring              483 non-null    object        
 6   Music by              514 non-null    object        
 7   Production company    210 non-null    object        
 8   Distributed by        521 non-null    object        
 9   Release dates         187 non-null    object        
 10  Running time          507 non-null    object        
 11  Country               463 non-null    object        
 12  Language              498 non-null    object        
 13  Budget              